In [1]:
import rust_machine_learning
import pandas as pd
from syed_ml_lib import *

In [2]:
import os
# TODO: Think about supporting audio as well, since CNNs are also used for speech recognition

# EXPECTED DIRECTORY STRUCTURE
# image_directory
#   classification_1
#       all images of classification_1 in jpg format here
#   classification_2
#       all images of classification_2 in jpg format here

# Collect images into a dataframe, with their corresponding labels
image_directory = "../data/animals-10/"
image_directory_path = os.path.abspath(image_directory)

# Returns each image with its metadata. Raw Image: Vec<Vec<Vec<f32>>>, Metadata: Vec<(label, width, height)>
#                                                  Example->depth->data


batch_size = 32 # Set to 0 to load all images
batch_image_loader = rust_machine_learning.ImageBatchLoader(image_directory_path, 128,128)


import time
start_time = time.perf_counter()
num_iter = 50
for i in range(0,num_iter):
    images, meta_data = batch_image_loader.batch_sample(batch_size)

end_time = time.perf_counter()

elapsed_time = (end_time - start_time) / num_iter
print(f"Execution time from python: {elapsed_time} seconds")  # Print with high resolution

# del images # Prevents memory leak (Notebook keep it around...)
len(meta_data), (len(images), len(images[0]), len(images[0][0])), meta_data[0], 

Reading images for classification "pecora"
Reading images for classification "cane"
Reading images for classification "ragno"
Reading images for classification "cavallo"
Reading images for classification "gallina"
Reading images for classification "scoiattolo"
Reading images for classification "farfalla"
Reading images for classification "gatto"
Reading images for classification "elefante"
Reading images for classification "mucca"
Time to load samples: 16
Time to load samples: 6
Time to load samples: 12
Time to load samples: 7
Time to load samples: 7
Time to load samples: 238
Time to load samples: 7
Time to load samples: 7
Time to load samples: 8
Time to load samples: 6
Time to load samples: 6
Time to load samples: 10
Time to load samples: 7
Time to load samples: 7
Time to load samples: 6
Time to load samples: 9
Time to load samples: 9
Time to load samples: 6
Time to load samples: 7
Time to load samples: 8
Time to load samples: 7
Time to load samples: 5
Time to load samples: 7
Time to 

(32, (32, 3, 16384), 'elefante')

In [16]:
import numpy as np
view_image(np.array(images[14][0]), 128, 128, 'gray')

In [3]:
# Prepare data for training with input rust functions expect
label_col = 0

# Normalize columns
labels = encoded.iloc[:, label_col]
max_per_col = encoded.abs().max().replace(0,1)
encoded = (encoded / max_per_col)
encoded.iloc[:, label_col] = labels

train_data, test_data = split_train_test(encoded, 0.5, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()

# Data Correlation
# corr = train_data_features.corr()
# fig = px.imshow(corr, aspect="auto", origin='lower')
# fig.show()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
test_data_labels = test_data.iloc[:,label_col].values.tolist()
train_data_features

NameError: name 'encoded' is not defined

In [ ]:
# Just showing one of the images as an example
row = 0
img_data = train_data_features.iloc[row].values
classification = train_data_labels[row]
print("Classification for row " + str(row) + " is: " + str(classification))
view_image(img_data, 28, 28, 'gray')

Classification for row 0 is: 0.0


In [ ]:
# Now train the model
hidden_layer_sizes = [10]
num_classifications = 10
learning_rate = 1e-2
num_iterations = 5000
batch_size = 64
trained_model = rust_machine_learning.BasicNeuralNetwork(train_data_features_list, train_data_labels, hidden_layer_sizes, num_classifications, learning_rate, num_iterations, batch_size)

In [ ]:
# Now let us test the model
labelled_results = trained_model.classify(test_data_features_list)

num_correct = 0
tolerance = 0.05
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", 100.0 * num_correct / len(test_data_labels))

Percent Correct:  51.26


In [ ]:
# Interactive Test
import random
row = random.randint(0, len(test_data_features))
img_data = test_data_features.iloc[row].values
list_img_data = [test_data_features_list[row]]
prediction = trained_model.classify(list_img_data)[0]
print(prediction)
view_image(img_data, 28, 28, 'gray')

9.0
